[![Roboflow Notebooks](https://ik.imagekit.io/roboflow/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# Vector Analysis with Scikit-learn and Bokeh

---

[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow-blogpost.svg)](https://blog.roboflow.com/vector-analysis)
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/roboflow/notebooks)

Vector embeddings have become a hot topic in machine learning these days. Roboflow uses [multimodal CLIP embeddings](https://blog.roboflow.com/openai-clip/) embeddings to power our [dataset search](https://blog.roboflow.com/dataset-search/) feature. And, now, you can access your dataset's CLIP embeddings via the Roboflow API.

In this notebook, we will load embeddings for a dataset from [Roboflow](https://roboflow.com), analyze them with [the t-SNE algorithm](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) using [Scikit-learn](https://scikit-learn.org/stable/) and visualize them in an interactive plot using [Bokeh](https://bokeh.org/).

**Let's begin!**

In [1]:
# Install dependencies
!pip install -qqq --upgrade roboflow sklearn pandas bokeh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 862.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.14.4 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.1.0 which is incompatible.
google-colab 1.0.0 requires pandas~=1.5.3, but you have pandas 2.0.1 which is incompatible.

In [2]:
# Login with your Roboflow account
import roboflow
roboflow.login()

visit https://app.roboflow.com/auth-cli to get your authentication token.
Paste the authentication token here: ··········


## Finding a Dataset

If you already have a custom dataset loaded into Roboflow, you can analyze it by pasting the link in the `roboflow_url` field below. Otherwise, find an open source dataset on [Roboflow Universe](https://universe.roboflow.com) to use (there are over 200,000 to choose from).

This notebook will work best if you use an `object-detection` or `segmentation` dataset since we will utilize their annotations.

In [3]:
#@title Pull Data from Roboflow API
roboflow_url = "https://universe.roboflow.com/april-public-yibrz/never-gonna" #@param {type:"string"}
image_limit = 1500 #@param {type: "number"}
closest_to_text = "random" #@param {type:"string"}

import re
(workspace, project) = (m.group(1), m.group(2)) if (m := re.match(r"https:\/\/(?:app|universe|api)\.roboflow\.com\/([^/]+)\/([^/]+)", roboflow_url)) else (None, None)

print(workspace, project)

import requests

offset = 0
images = []

project_url = f'https://api.roboflow.com/{workspace}/{project}'
print(f'Loading Project Info from: {project_url}')
response = requests.get(
    project_url,
    headers={
        'Authorization': f'Bearer {roboflow.load_roboflow_api_key()}'
    }
)

if response.status_code != 200:
    print(f"Error: {response.status_code}. {response.json().get('error', 'No additional information available.')}")
    assert(response.status_code == 200)

project_info = response.json()
annotation_name = project_info.get("project").get("annotation")

search_url = f'https://api.roboflow.com/{workspace}/{project}/search'
print(f'Loading Embeddings via Roboflow Search from: {search_url}')
print('This can take a little while if the dataset is large.')

while True:
    response = requests.post(
        search_url,
        json={
            'prompt': closest_to_text,
            'limit': 250,
            'offset': offset,
            'fields': ['id', 'owner', 'name', 'split', 'annotations', 'embedding']
        },
        headers={
            'Authorization': f'Bearer {roboflow.load_roboflow_api_key()}'
        }
    )

    if response.status_code != 200:
        print(f"Error: {response.status_code}. {response.json().get('error', 'No additional information available.')}")
        break

    results = response.json().get('results', [])

    numResults = len(results)
    offset = offset + numResults

    images = images + results

    if numResults == 0 or len(images) >= min(image_limit, 10000):
        break

for image in images:
    image['thumbnail_url'] = f"https://storage.googleapis.com/roboflow-platform-sources/{image['owner']}/{image['id']}/thumb.jpg"
    image['annotation_url'] = f"https://storage.googleapis.com/roboflow-platform-sources/{image['owner']}/{image['id']}/annotation-{annotation_name}.png"

print(f"Got {len(images)} total embeddings.")

april-public-yibrz never-gonna
Loading Project Info from: https://api.roboflow.com/april-public-yibrz/never-gonna
Loading Embeddings via Roboflow Search from: https://api.roboflow.com/april-public-yibrz/never-gonna/search
This can take a little while if the dataset is large.
Got 1500 total embeddings.


## Vector Analysis

We've now loaded our CLIP vectors into the notebook memory using the Roboflow Search API (click `Show code` above to see how we did that). Now we'll use t-SNE to reduce their dimensionality into something we can visualize on your screen.

By default, we'll analyze all of the images in the Roboflow project. If you only want to show `annotated` images that have been added to the dataset, switch from `all` to `only_annotated` below.

In [4]:
#@title Fit a t-SNE Plot with sklearn
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd

images_to_use = "all" #@param ["only_annotated", "all"]

if images_to_use == "all":
  filtered_images = images
else:
  # filter out images not yet added to a train/test split (still in annotation)
  filtered_images = [image for image in images if image['split'] != 'none']

tsne = TSNE(n_components=2, verbose=1)
z = tsne.fit_transform(np.array([image['embedding'] for image in filtered_images]))

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1500 samples in 0.001s...
[t-SNE] Computed neighbors for 1500 samples in 0.153s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1500
[t-SNE] Computed conditional probabilities for sample 1500 / 1500
[t-SNE] Mean sigma: 3.952271
[t-SNE] KL divergence after 250 iterations with early exaggeration: 58.396965
[t-SNE] KL divergence after 1000 iterations: 0.459478


## Visualization

Now that we've analyzed the vectors, let's visualize them! We can choose to color our scatterplot by the classes, the number of objects in the image, or their train/valid/test split. You can hover over individual data points to see the image it represents.

You can use a plot like this to look for patterns and outliers in your dataset. For example, if your train/valid/test set are very disjoint, it might mean they're not representative & your model may struggle. If you see an instance of a class all out by itself it might represent an edge-case or a labeling error.

In [5]:
#@title Visualize with Bokeh
from collections import defaultdict
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource, CustomJS, Button, CategoricalColorMapper, Div, ImageURL
from bokeh.layouts import column, row
from bokeh.palettes import Category10, Category20, Turbo256
from bokeh.io import output_notebook, output_file, save
from bokeh.util.browser import view
from math import ceil, floor

df = pd.DataFrame()

color_by = "classes in image" #@param ["classes in image", "object count", "train/valid/test split"]

def get_highest_value_key(d):
    return max(d, key=d.get)

def get_color_palette(num_classes):
    if num_classes <= 10:
      return Category10[10]
    elif num_classes <= 20:
        return Category20[20]
    else:
        return [Turbo256[i] for i in range(0, 256, int(ceil(256 / num_classes)))][::-1]

def count_classes(images):
    class_count = defaultdict(int)
    for image in images:
        if 'classes' in image['annotations']:
            for cls, count in image['annotations']['classes'].items():
                class_count[cls] += count
    return class_count

if color_by == "classes in image":
    color_by_list = [
      ",".join(sorted(image['annotations']['classes'].keys(), key=lambda k: (-image['annotations']['classes'][k], k)))
      if 'classes' in image['annotations'] and len(image['annotations']['classes']) > 0
      else ('null' if 'classes' in image['annotations'] else 'unannotated')
      for image in filtered_images
    ]
    df["caption"] = color_by_list

    unique_combinations = list(set(color_by_list))
    if len(unique_combinations) > 20:
        total_class_count = count_classes(filtered_images)

        # Change strategy to group by the key with the lowest total count
        color_by_list = [
            min(image['annotations']['classes'], key=lambda k: (total_class_count[k], k))
            if 'classes' in image['annotations'] and len(image['annotations']['classes']) > 0
            else ('null' if 'classes' in image['annotations'] else 'unannotated')
            for image in filtered_images
        ]

    df["color_by"] = color_by_list

if color_by == "object count":
  # Visualize by object count
  df["color_by"] = [str(sum(image['annotations']['classes'].values())) if 'classes' in image['annotations'] and len(image['annotations']['classes']) > 0 else ('0' if 'classes' in image['annotations'] else 'unannotated') for image in filtered_images]
  df["caption"] = df["color_by"]

if color_by == "train/valid/test split":
  # Visualize by train/valid/test split
  df["color_by"] = [image.get("split", "none") for image in filtered_images]
  df["caption"] = df["color_by"]

df["x"] = z[:, 0]
df["y"] = z[:, 1]
df["thumbnail_url"] = [image['thumbnail_url'] for image in filtered_images]
df["annotation_url"] = [image['annotation_url'] for image in filtered_images]

def visualize(df):
  output_notebook()

  p = figure(title=f"{workspace}/{project} by {color_by}", height=600, sizing_mode='stretch_width')

  renderers = []

  # Sort the unique values in the 'color_by' column
  unique_values = df['color_by'].unique()
  if all(val.isnumeric() for val in unique_values if val != 'unannotated'):
      unique_values = sorted(unique_values, key=lambda x: (x != 'unannotated', int(x) if x.isnumeric() else x))
  else:
      unique_values = df['color_by'].value_counts().index.tolist()

  if 'null' in unique_values:
      unique_values.remove('null')
      unique_values.insert(0, 'null')
      
  if 'unannotated' in unique_values:
      unique_values.remove('unannotated')
      unique_values.insert(0, 'unannotated')

  num_categories = len(unique_values)
  color_palette = get_color_palette(num_categories)

  # Color into groups (according to the color_by column)
  for category, color in zip(unique_values, color_palette):
      mask = df['color_by'] == category
      source = ColumnDataSource(df.loc[mask])
      
      r = p.scatter(x='x', y='y', source=source, color=color, size=7)
      renderers.append(r)

  # Show the image when you hover over it
  hover = HoverTool()
  p.add_tools(hover)

  hover.tooltips = """
      <div>
        @caption
        <div style="
          position: relative;
          width: 200px;
          height: 200px;
          margin: 10px;
          background-size: cover;
          background-position: center center;
          background-image: url(@annotation_url), url(@thumbnail_url);
        "></div>
      </div>
  """

  # Add a Show All button
  js_show_all_code = """
      for (var i = 0; i < renderers.length; i++) {
          renderers[i].visible = true;
      }
  """
  show_all_button = Button(label="Show All", button_type="success", width=80)
  show_all_button.js_on_click(CustomJS(args={'renderers': renderers}, code=js_show_all_code))

  # Add a button to isolate down to each category
  js_show_single_code = """
      for (var i = 0; i < renderers.length; i++) {
          if (i == index) {
              renderers[i].visible = true;
          } else {
              renderers[i].visible = false;
          }
      }
  """
  category_buttons = []
  for i, (category, color) in enumerate(zip(unique_values, color_palette)):
      category_button = Button(label=category, button_type="default", css_classes=[f'custom-button-{i}'], width=80, height=40)
      category_button.name = f"category-button-{i}"
      
      js_callback = CustomJS(args={'renderers': renderers, 'index': i}, code=js_show_single_code)
      category_button.js_on_click(js_callback)
      
      category_buttons.append(category_button)

  # Add CSS to style the buttons
  css = '''
  .bk.custom-button {
      margin: 0;
      padding: 6px 12px;
  }
  '''
  for i, (button, color) in enumerate(zip(category_buttons, color_palette)):
      css += f"""
      .bk.custom-button-{i} {{
          height: 32px !important;
      }}
      .bk.custom-button-{i} .bk-btn-default {{
          background-color: {color};
          border-color: {color};
          color: white;
          text-shadow: 0 0 3px rgba(0, 0, 0, 0.5);
      }}
      """
  style = Div(text=f'<style>{css}</style>')

  def chunk(lst, n):
      """Yield successive n-sized chunks from lst."""
      for i in range(0, len(lst), n):
          yield lst[i:i + n]

  # Split category_buttons into groups of 10
  category_buttons_grouped = list(chunk(category_buttons, 10))

  # Add the buttons in rows
  buttons_layout = column([row([show_all_button])] + [row(group) for group in category_buttons_grouped])

  # Show the visualization
  layout = column(style, p, buttons_layout, sizing_mode='stretch_width')
  show(layout)

visualize(df)